<a href="https://colab.research.google.com/github/gayun99/esaa21/blob/main/%EB%AA%A8%EB%8D%B8%ED%9B%88%EB%A0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **4. 모델 훈련 연습문제**


### 1) 수백만 개의 특성을 가진 훈련 세트에서는 어떤 선형 회귀 알고리즘을 사용할 수 있을까요?

수백만 개의 특성이 있는 훈련세트는 확률적 경사 하강법이나 미니배치 경사 하강법을 사용할 수 있다. 훈련 세트가 메모리 크기에 맞으면 배치 경사 하강법도 사용가능하다.



### 9) 릿지 회귀를 사용했을 때 훈련 오차와 검증 오차가 거의 비슷하고 둘 다 높았습니다. 이 모델에는 높은 편향이 문제인가요, 아니면 높은 분산이 문제인가요? 규제 하이퍼파라미터 α 를 증가시켜야할까요, 아니면 줄여야 할까요?

검증 오차와 훈련 오차가 비슷하고 매우 높다면 과소적합되었을 가능성이 있다. 즉 높은 편향이 문제이므로 규제 하이퍼파라미터 a를 감소시켜야 한다.



### 10) 다음과 같이 사용해야 하는 이유는?

- 평범한 선형 회귀(즉, 아무런 규제가 없는 모델) 대신 릿지 회귀

- 릿지 회귀 대신 라쏘 회귀

- 라쏘 회귀 대신 엘라스틱넷

1. 규제가 있는 릿지 회귀가 규제가 없는 평범한 선형회귀보다 성능이 좋으므로 

2. 라쏘회귀는 가중치를 0으로 만들 수 있어 변수선택의 효과를 가지므로 몇 개의 특성만 유효하다고 판단될 때 사용하는 것이 좋다.

3. 라쏘는 몇몇 특성이 강하게 연관되어있거나 훈련 샘플보다 특성이 더 많을 때 불규칙하게 행동한다. 따라서 엘라스틱 넷을 라쏘보다 선호한다. 



### 12) 다음 사이트에 들어가서 연습문제 12번 일부를 필사하세요.

https://github.com/rickiepark/handson-ml2/blob/master/04_training_linear_models.ipynb

- 67번 셀 부터 80번 셀 까지 필사하시면 됩니다.

- 시간이 남는다면 그 이후 코드까지 필사하셔도 좋습니다.

- 수식이 설명되어있는 부분을 보고싶으면 그 부분을 그대로 복사하여 텍스트 창에 붙여넣기 한 다음 직접 실행시키시면 확인해볼 수 있습니다.

In [1]:
from sklearn import datasets
iris = datasets.load_iris()
list(iris.keys())

['data', 'target', 'target_names', 'DESCR', 'feature_names', 'filename']

In [2]:
X = iris["data"][:, (2,3)]
y = iris["target"]

In [3]:
#모든 샘플에 편향 추가
import numpy as np
X_with_bias = np.c_[np.ones([len(X),1]), X]

In [4]:
np.random.seed(2042)

In [9]:
test_ratio = 0.2
validation_ratio = 0.2
total_size = len(X_with_bias)

test_size = int(total_size * test_ratio)
validation_size = int(total_size * validation_ratio)
train_size = total_size - test_size - validation_size

rnd_indices = np.random.permutation(total_size)

X_train = X_with_bias[rnd_indices[:train_size]]
y_train = y[rnd_indices[:train_size]]
X_valid = X_with_bias[rnd_indices[train_size:-test_size]]
y_valid = y[rnd_indices[train_size:-test_size]]
X_test = X_with_bias[rnd_indices[-test_size:]]
y_test = y[rnd_indices[-test_size:]]

In [10]:
def to_one_hot(y):
    n_classes = y.max() + 1
    m = len(y)
    Y_one_hot = np.zeros((m, n_classes))
    Y_one_hot[np.arange(m), y] = 1
    return Y_one_hot

In [11]:
y_train[:10]

array([0, 1, 2, 1, 1, 0, 1, 1, 1, 0])

In [12]:
to_one_hot(y_train[:10])

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [13]:
Y_train_one_hot = to_one_hot(y_train)
Y_valid_one_hot = to_one_hot(y_valid)
Y_test_one_hot = to_one_hot(y_test)

In [14]:
def softmax(logits):
    exps = np.exp(logits)
    exp_sums = np.sum(exps, axis=1, keepdims=True)
    return exps / exp_sums

In [15]:
n_inputs = X_train.shape[1]
n_outputs = len(np.unique(y_train))

In [16]:
eta = 0.01
n_iterations = 5001
m = len(X_train)
epsilon = 1e-7

Theta = np.random.randn(n_inputs, n_outputs)

for iteration in range(n_iterations):
    logits = X_train.dot(Theta)
    Y_proba = softmax(logits)
    if iteration % 500 == 0:
        loss = -np.mean(np.sum(Y_train_one_hot * np.log(Y_proba + epsilon), axis=1))
        print(iteration, loss)
    error = Y_proba - Y_train_one_hot
    gradients = 1/m * X_train.T.dot(error)
    Theta = Theta - eta * gradients

0 5.446205811872683
500 0.8350062641405651
1000 0.6878801447192402
1500 0.6012379137693314
2000 0.5444496861981872
2500 0.5038530181431525
3000 0.47292289721922487
3500 0.44824244188957774
4000 0.42786510939287936
4500 0.41060071429187134
5000 0.3956780375390374


In [17]:
Theta

array([[ 3.32094157, -0.6501102 , -2.99979416],
       [-1.1718465 ,  0.11706172,  0.10507543],
       [-0.70224261, -0.09527802,  1.4786383 ]])

In [18]:
logits = X_valid.dot(Theta)
Y_proba = softmax(logits)
y_predict = np.argmax(Y_proba, axis=1)

accuracy_score = np.mean(y_predict == y_valid)
accuracy_score

0.9666666666666667

In [19]:
eta = 0.1
n_iterations = 5001
m = len(X_train)
epsilon = 1e-7
alpha = 0.1 
Theta = np.random.randn(n_inputs, n_outputs)

for iteration in range(n_iterations):
    logits = X_train.dot(Theta)
    Y_proba = softmax(logits)
    if iteration % 500 == 0:
        xentropy_loss = -np.mean(np.sum(Y_train_one_hot * np.log(Y_proba + epsilon), axis=1))
        l2_loss = 1/2 * np.sum(np.square(Theta[1:]))
        loss = xentropy_loss + alpha * l2_loss
        print(iteration, loss)
    error = Y_proba - Y_train_one_hot
    gradients = 1/m * X_train.T.dot(error) + np.r_[np.zeros([1, n_outputs]), alpha * Theta[1:]]
    Theta = Theta - eta * gradients

0 6.629842469083912
500 0.5339667976629505
1000 0.5036400750148942
1500 0.49468910594603216
2000 0.4912968418075476
2500 0.48989924700933296
3000 0.4892990598451198
3500 0.48903512443978603
4000 0.4889173621830818
4500 0.48886433374493027
5000 0.48884031207388184


In [20]:
logits = X_valid.dot(Theta)
Y_proba = softmax(logits)
y_predict = np.argmax(Y_proba, axis=1)

accuracy_score = np.mean(y_predict == y_valid)
accuracy_score

1.0